In [1]:
class GrafoLA:
    def __init__(self):
        # Usaremos um dicionário para mapear rótulos e os pesos de nós usaremos tuplas
        # para suas listas de adjacência.
        self.adj = {}
        # Para controlar os nós visitados em algoritmos como Busca em Profundidade (DFS) e Busca em Largura (BFS),
        # usaremos um dicionário que mapeia o rótulo do nó para um status (0 para não visitado, 1 para visitado).
        self.__visitado = {}
        self.ordem_visitados = []

    def __str__(self):
        # Retorna uma representação legível do grafo.
        # Ex: "A: ['B', 'C']\nB: ['A']"
        return '\n'.join([f"{no}: {vizinhos}" for no, vizinhos in self.adj.items()])

    def adicionar_vertice(self, n1):
        if n1 not in self.adj:
            self.adj[n1] = []
            
    def adicionar_aresta(self, n1, n2, peso=1, bidirecional=True):
        # Garante que os nós existam no dicionário de adjacência
        if n1 not in self.adj:
            self.adj[n1] = []
        if n2 not in self.adj:
            self.adj[n2] = []

        # Adiciona a aresta de n1 para n2
        self.adj[n1].append((n2, peso))

        # Se for bidirecional, adiciona a aresta de n2 para n1
        if bidirecional:
            self.adj[n2].append((n1, peso))

    def obter_vizinhos(self, n):
        # Retorna a lista de vizinhos de um nó.
        # Retorna uma lista vazia se o nó não existir.
        return self.adj.get(n, [])

    # Comum para DFS e BFS
    def __visitar(self, no): # 'node' -> 'no'
        # Marca um nó como visitado e o adiciona à ordem de visita.
        self.__visitado[no] = 1
        self.ordem_visitados.append(no)

    # DFS - Busca em Profundidade
    def __dfs(self, n):
        # Implementação recursiva da Busca em Profundidade (DFS).
        self.__visitar(n)
        for vizinho_e_peso in self.obter_vizinhos(n):
            vizinho = vizinho_e_peso[0]
            # Verifica se o vizinho não foi visitado.
            if self.__visitado.get(vizinho, 0) == 0:
                self.__dfs(vizinho)

    def DFS(self, no_inicial):
        # Inicia a Busca em Profundidade (DFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós para cada nova execução de DFS.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        if no_inicial in self.adj:
            self.__dfs(no_inicial)
        return self.ordem_visitados
    # DFS - Busca em profundidade

    # BFS - Busca em largura
    def BFS(self, no_inicial):
        # Inicia a Busca em Largura (BFS) a partir de um nó.
        # Redefine o status de visitado para todos os nós.
        self.__visitado = {no: 0 for no in self.adj}
        self.ordem_visitados = []
        fila = []  # Usaremos uma lista como fila para a BFS

        if no_inicial not in self.adj:
            return self.ordem_visitados

        self.__visitar(no_inicial)
        fila.append(no_inicial)

        while fila:
            no_atual = fila.pop(0)  # Pega o primeiro nó da fila
            for vizinho_e_peso in self.obter_vizinhos(no_atual):
                vizinho = vizinho_e_peso[0]
                if self.__visitado.get(vizinho, 0) == 0:
                    self.__visitar(vizinho)
                    fila.append(vizinho)
        return self.ordem_visitados
    # BFS - Busca em largura

    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos
    def SSSP(self, no_inicial, limite=None):
        if no_inicial not in self.adj:
            return {}
        
        return self.__sssp(no_inicial, limite)
    
    def __sssp(self, no_inicial, limite):
        caminhos = {no_inicial: [no_inicial]}
        fila = [(no_inicial, 0)] # (no, nível) . O nível serve para que o limite possa ver se o nó que vou visitar é permitido

        while fila:
            no_atual, nivel = fila.pop(0)

            if limite is not None and nivel >= limite:
                continue
            
            for vizinho_e_peso in self.obter_vizinhos(no_atual):
                vizinho = vizinho_e_peso[0]
                if vizinho not in caminhos:
                    caminhos[vizinho] = caminhos[no_atual] + [vizinho]
                    fila.append((vizinho, nivel + 1))
        return caminhos
    # Algoritmo para encontrar o menor caminho de um nó determinado para todos os outros, em grafos sem pesos

    # dijkstra
    def DIJKSTRA(self, no_inicial):
        import math
        distancias = {no: math.inf for no in self.adj}
        distancias[no_inicial] = 0

        predecessores = {no: None for no in self.adj}
        
        nos_visitados = set()

        # (0, no_inicial) = (distancia_atual_ao_no, nó)
        fila_prioridade = [(0, no_inicial)]
        
        while fila_prioridade:
            dist_atual, no_atual = fila_prioridade.pop(fila_prioridade.index(min(fila_prioridade)))
            
            if no_atual in nos_visitados:
                continue

            nos_visitados.add(no_atual)

            for vizinho_e_peso in self.obter_vizinhos(no_atual):
                vizinho, peso = vizinho_e_peso

                if vizinho in nos_visitados:
                    continue
                
                nova_distancia = dist_atual + peso

                if nova_distancia < distancias[vizinho]:
                    distancias[vizinho] = nova_distancia
                    predecessores[vizinho] = no_atual
                    fila_prioridade.append((nova_distancia, vizinho))

        caminhos_finais = {}
        for no_destino in self.adj:
            if distancias[no_destino] == math.inf:
                caminhos_finais[no_destino] = []
            else:
                caminho = []
                temp_no = no_destino
                while temp_no is not None:
                    caminho.insert(0, temp_no)
                    temp_no = predecessores[temp_no]
                caminhos_finais[no_destino] = caminho
        return distancias, caminhos_finais


In [2]:
# Dicionário de todas as capitais com suas siglas
capitais_com_siglas = {
    "Porto Alegre": "RS", "Florianópolis": "SC", "Curitiba": "PR", "São Paulo": "SP",
    "Rio de Janeiro": "RJ", "Belo Horizonte": "MG", "Vitória": "ES", "Campo Grande": "MS",
    "Cuiabá": "MT", "Goiânia": "GO", "Brasília": "DF", "Palmas": "TO",
    "Salvador": "BA", "Aracaju": "SE", "Maceió": "AL", "Recife": "PE",
    "João Pessoa": "PB", "Natal": "RN", "Fortaleza": "CE", "Teresina": "PI",
    "São Luís": "MA", "Belém": "PA", "Macapá": "AP", "Manaus": "AM",
    "Porto Velho": "RO", "Rio Branco": "AC", "Boa Vista": "RR"
}

# Função para determinar o peso com base na distância
def obter_peso_por_distancia(distancia_km):
    if 0 <= distancia_km <= 150:
        return 1
    elif 151 <= distancia_km <= 300:
        return 2
    elif 301 <= distancia_km <= 450:
        return 3
    elif 451 <= distancia_km <= 600:
        return 4
    elif 601 <= distancia_km <= 750:
        return 5
    elif 751 <= distancia_km <= 900:
        return 6
    elif 901 <= distancia_km <= 1050:
        return 7
    elif 1051 <= distancia_km <= 1200:
        return 8
    elif 1201 <= distancia_km <= 1350:
        return 9
    elif 1351 <= distancia_km <= 1500:
        return 10
    elif 1501 <= distancia_km <= 1650:
        return 11
    elif 1651 <= distancia_km <= 1800:
        return 12
    else:
        # Para distâncias maiores que 1800 km
        return 13

# Criando nomes dos nós com a sigla do estado
def nome_com_sigla(capital_nome):
    sigla = capitais_com_siglas.get(capital_nome, "")
    return f"{capital_nome} ({sigla})" if sigla else capital_nome

# Instanciando o grafo
grafo_capitais = GrafoLA()

# Adicionando todos os vértices (capitais com sigla)
for capital in capitais_com_siglas.keys():
    grafo_capitais.adicionar_vertice(nome_com_sigla(capital))

# Adicionando arestas com base nas fronteiras entre os estados.
# As distâncias são aproximadas para o cálculo do peso.

# Região Sul
grafo_capitais.adicionar_aresta(nome_com_sigla("Porto Alegre"), nome_com_sigla("Florianópolis"), obter_peso_por_distancia(470))
grafo_capitais.adicionar_aresta(nome_com_sigla("Florianópolis"), nome_com_sigla("Curitiba"), obter_peso_por_distancia(300))

# Conexões entre Sul e Sudeste
grafo_capitais.adicionar_aresta(nome_com_sigla("Curitiba"), nome_com_sigla("São Paulo"), obter_peso_por_distancia(400))
grafo_capitais.adicionar_aresta(nome_com_sigla("Curitiba"), nome_com_sigla("Campo Grande"), obter_peso_por_distancia(890))

# Região Sudeste
grafo_capitais.adicionar_aresta(nome_com_sigla("São Paulo"), nome_com_sigla("Rio de Janeiro"), obter_peso_por_distancia(430))
grafo_capitais.adicionar_aresta(nome_com_sigla("São Paulo"), nome_com_sigla("Belo Horizonte"), obter_peso_por_distancia(580))
grafo_capitais.adicionar_aresta(nome_com_sigla("Belo Horizonte"), nome_com_sigla("Rio de Janeiro"), obter_peso_por_distancia(441))
grafo_capitais.adicionar_aresta(nome_com_sigla("Belo Horizonte"), nome_com_sigla("Vitória"), obter_peso_por_distancia(514))
grafo_capitais.adicionar_aresta(nome_com_sigla("Rio de Janeiro"), nome_com_sigla("Vitória"), obter_peso_por_distancia(520))

# Conexões entre Sudeste e Centro-Oeste
grafo_capitais.adicionar_aresta(nome_com_sigla("São Paulo"), nome_com_sigla("Campo Grande"), obter_peso_por_distancia(984))
grafo_capitais.adicionar_aresta(nome_com_sigla("Belo Horizonte"), nome_com_sigla("Campo Grande"), obter_peso_por_distancia(1260))
grafo_capitais.adicionar_aresta(nome_com_sigla("Belo Horizonte"), nome_com_sigla("Goiânia"), obter_peso_por_distancia(891))
grafo_capitais.adicionar_aresta(nome_com_sigla("Belo Horizonte"), nome_com_sigla("Brasília"), obter_peso_por_distancia(739))

# Conexões entre Sudeste e Nordeste
grafo_capitais.adicionar_aresta(nome_com_sigla("Belo Horizonte"), nome_com_sigla("Salvador"), obter_peso_por_distancia(1426))
grafo_capitais.adicionar_aresta(nome_com_sigla("Vitória"), nome_com_sigla("Salvador"), obter_peso_por_distancia(1157))

# Região Centro-Oeste
grafo_capitais.adicionar_aresta(nome_com_sigla("Brasília"), nome_com_sigla("Goiânia"), obter_peso_por_distancia(200))
grafo_capitais.adicionar_aresta(nome_com_sigla("Campo Grande"), nome_com_sigla("Cuiabá"), obter_peso_por_distancia(700))
grafo_capitais.adicionar_aresta(nome_com_sigla("Goiânia"), nome_com_sigla("Campo Grande"), obter_peso_por_distancia(850))
grafo_capitais.adicionar_aresta(nome_com_sigla("Goiânia"), nome_com_sigla("Cuiabá"), obter_peso_por_distancia(887))

# Conexões entre Centro-Oeste e Nordeste
grafo_capitais.adicionar_aresta(nome_com_sigla("Goiânia"), nome_com_sigla("Salvador"), obter_peso_por_distancia(1646))

# Conexões Centro-Oeste com Norte
grafo_capitais.adicionar_aresta(nome_com_sigla("Cuiabá"), nome_com_sigla("Porto Velho"), obter_peso_por_distancia(1460))
grafo_capitais.adicionar_aresta(nome_com_sigla("Cuiabá"), nome_com_sigla("Palmas"), obter_peso_por_distancia(1499))
grafo_capitais.adicionar_aresta(nome_com_sigla("Cuiabá"), nome_com_sigla("Belém"), obter_peso_por_distancia(2647))
grafo_capitais.adicionar_aresta(nome_com_sigla("Goiânia"), nome_com_sigla("Palmas"), obter_peso_por_distancia(800))

# Região Nordeste
grafo_capitais.adicionar_aresta(nome_com_sigla("Salvador"), nome_com_sigla("Aracaju"), obter_peso_por_distancia(350))
grafo_capitais.adicionar_aresta(nome_com_sigla("Salvador"), nome_com_sigla("Maceió"), obter_peso_por_distancia(578))
grafo_capitais.adicionar_aresta(nome_com_sigla("Salvador"), nome_com_sigla("Recife"), obter_peso_por_distancia(805))
grafo_capitais.adicionar_aresta(nome_com_sigla("Salvador"), nome_com_sigla("Teresina"), obter_peso_por_distancia(1156))
grafo_capitais.adicionar_aresta(nome_com_sigla("Aracaju"), nome_com_sigla("Maceió"), obter_peso_por_distancia(270))
grafo_capitais.adicionar_aresta(nome_com_sigla("Maceió"), nome_com_sigla("Recife"), obter_peso_por_distancia(256))
grafo_capitais.adicionar_aresta(nome_com_sigla("Recife"), nome_com_sigla("João Pessoa"), obter_peso_por_distancia(116))
grafo_capitais.adicionar_aresta(nome_com_sigla("Recife"), nome_com_sigla("Fortaleza"), obter_peso_por_distancia(777))
grafo_capitais.adicionar_aresta(nome_com_sigla("João Pessoa"), nome_com_sigla("Natal"), obter_peso_por_distancia(181))
grafo_capitais.adicionar_aresta(nome_com_sigla("João Pessoa"), nome_com_sigla("Fortaleza"), obter_peso_por_distancia(670))
grafo_capitais.adicionar_aresta(nome_com_sigla("Natal"), nome_com_sigla("Fortaleza"), obter_peso_por_distancia(521))
grafo_capitais.adicionar_aresta(nome_com_sigla("Fortaleza"), nome_com_sigla("Teresina"), obter_peso_por_distancia(181))
grafo_capitais.adicionar_aresta(nome_com_sigla("Teresina"), nome_com_sigla("São Luís"), obter_peso_por_distancia(432))

# Região Norte
grafo_capitais.adicionar_aresta(nome_com_sigla("Belém"), nome_com_sigla("Palmas"), obter_peso_por_distancia(1210))
grafo_capitais.adicionar_aresta(nome_com_sigla("Belém"), nome_com_sigla("Macapá"), obter_peso_por_distancia(528))
grafo_capitais.adicionar_aresta(nome_com_sigla("Porto Velho"), nome_com_sigla("Manaus"), obter_peso_por_distancia(888))
grafo_capitais.adicionar_aresta(nome_com_sigla("Porto Velho"), nome_com_sigla("Rio Branco"), obter_peso_por_distancia(509))
grafo_capitais.adicionar_aresta(nome_com_sigla("Rio Branco"), nome_com_sigla("Manaus"), obter_peso_por_distancia(1396))
grafo_capitais.adicionar_aresta(nome_com_sigla("Manaus"), nome_com_sigla("Belém"), obter_peso_por_distancia(3060))
grafo_capitais.adicionar_aresta(nome_com_sigla("Manaus"), nome_com_sigla("Boa Vista"), obter_peso_por_distancia(747))
grafo_capitais.adicionar_aresta(nome_com_sigla("Boa Vista"), nome_com_sigla("Belém"), obter_peso_por_distancia(2815))

# Conexões entre Norte e Nordeste
grafo_capitais.adicionar_aresta(nome_com_sigla("Belém"), nome_com_sigla("São Luís"), obter_peso_por_distancia(576))
grafo_capitais.adicionar_aresta(nome_com_sigla("Palmas"), nome_com_sigla("São Luís"), obter_peso_por_distancia(1253))
grafo_capitais.adicionar_aresta(nome_com_sigla("Palmas"), nome_com_sigla("Teresina"), obter_peso_por_distancia(1113))
grafo_capitais.adicionar_aresta(nome_com_sigla("Palmas"), nome_com_sigla("Salvador"), obter_peso_por_distancia(1438))

print("Grafo das Capitais Brasileiras:")
print(grafo_capitais)

Grafo das Capitais Brasileiras:
Porto Alegre (RS): [('Florianópolis (SC)', 4)]
Florianópolis (SC): [('Porto Alegre (RS)', 4), ('Curitiba (PR)', 2)]
Curitiba (PR): [('Florianópolis (SC)', 2), ('São Paulo (SP)', 3), ('Campo Grande (MS)', 6)]
São Paulo (SP): [('Curitiba (PR)', 3), ('Rio de Janeiro (RJ)', 3), ('Belo Horizonte (MG)', 4), ('Campo Grande (MS)', 7)]
Rio de Janeiro (RJ): [('São Paulo (SP)', 3), ('Belo Horizonte (MG)', 3), ('Vitória (ES)', 4)]
Belo Horizonte (MG): [('São Paulo (SP)', 4), ('Rio de Janeiro (RJ)', 3), ('Vitória (ES)', 4), ('Campo Grande (MS)', 9), ('Goiânia (GO)', 6), ('Brasília (DF)', 5), ('Salvador (BA)', 10)]
Vitória (ES): [('Belo Horizonte (MG)', 4), ('Rio de Janeiro (RJ)', 4), ('Salvador (BA)', 8)]
Campo Grande (MS): [('Curitiba (PR)', 6), ('São Paulo (SP)', 7), ('Belo Horizonte (MG)', 9), ('Cuiabá (MT)', 5), ('Goiânia (GO)', 6)]
Cuiabá (MT): [('Campo Grande (MS)', 5), ('Goiânia (GO)', 6), ('Porto Velho (RO)', 10), ('Palmas (TO)', 10), ('Belém (PA)', 13)]
Goiâ

In [7]:
grafo_capitais.DIJKSTRA('Rio de Janeiro (RJ)')

({'Porto Alegre (RS)': 12,
  'Florianópolis (SC)': 8,
  'Curitiba (PR)': 6,
  'São Paulo (SP)': 3,
  'Rio de Janeiro (RJ)': 0,
  'Belo Horizonte (MG)': 3,
  'Vitória (ES)': 4,
  'Campo Grande (MS)': 10,
  'Cuiabá (MT)': 15,
  'Goiânia (GO)': 9,
  'Brasília (DF)': 8,
  'Palmas (TO)': 15,
  'Salvador (BA)': 12,
  'Aracaju (SE)': 15,
  'Maceió (AL)': 16,
  'Recife (PE)': 18,
  'João Pessoa (PB)': 19,
  'Natal (RN)': 21,
  'Fortaleza (CE)': 22,
  'Teresina (PI)': 20,
  'São Luís (MA)': 23,
  'Belém (PA)': 24,
  'Macapá (AP)': 28,
  'Manaus (AM)': 31,
  'Porto Velho (RO)': 25,
  'Rio Branco (AC)': 29,
  'Boa Vista (RR)': 36},
 {'Porto Alegre (RS)': ['Rio de Janeiro (RJ)',
   'São Paulo (SP)',
   'Curitiba (PR)',
   'Florianópolis (SC)',
   'Porto Alegre (RS)'],
  'Florianópolis (SC)': ['Rio de Janeiro (RJ)',
   'São Paulo (SP)',
   'Curitiba (PR)',
   'Florianópolis (SC)'],
  'Curitiba (PR)': ['Rio de Janeiro (RJ)', 'São Paulo (SP)', 'Curitiba (PR)'],
  'São Paulo (SP)': ['Rio de Janeiro (R